In [199]:
import glob
import os
import random
import monai
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from tqdm.notebook import tqdm
import wandb

In [200]:
data_path = r'/home/jovyan/deeplearning'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class LoadData(monai.transforms.Transform):
    """
    This custom Monai transform loads the data from the segmentation dataset.
    Defining a custom transform is simple; just overwrite the __init__ function and __call__ function.
    """
    def __init__(self, keys=None):
        pass

    def __call__(self, sample):
        image = np.load(sample['img'])
        mask = np.load(sample['mask'])

        return {'img': image, 'mask': mask, 'img_meta_dict': {'affine': np.eye(2)}, 
                'mask_meta_dict': {'affine': np.eye(2)}}


In [201]:
segmentLV = True
segmentMYO = False
segmentRV = False

if segmentLV:
    train_transform = monai.transforms.Compose(
    [
        LoadData(),
        monai.transforms.ThresholdIntensityd(keys=['mask'],threshold=2.5, above=True, cval=0),
        monai.transforms.AddChanneld(keys=['img', 'mask']), # add 1 dimension because we have only 1 channel
        monai.transforms.ScaleIntensityd(keys=['img', 'mask'],minv=0, maxv=1) # scale intensity to values between 0-1
    ]
)

if segmentMYO:
    train_transform = monai.transforms.Compose(
    [
        LoadData(),
        monai.transforms.ThresholdIntensityd(keys=['mask'],threshold=2.5, above=False, cval=0),
        monai.transforms.ThresholdIntensityd(keys=['mask'],threshold=1.5, above=True, cval=0),
        monai.transforms.AddChanneld(keys=['img', 'mask']), # add 1 dimension because we have only 1 channel
        monai.transforms.ScaleIntensityd(keys=['img'],minv=0, maxv=1) # scale intensity to values between 0-1
    ]
)
    
if segmentRV:
    train_transform = monai.transforms.Compose(
    [
        LoadData(),
        monai.transforms.ThresholdIntensityd(keys=['mask'],threshold=1.5, above=False, cval=0),
        monai.transforms.AddChanneld(keys=['img', 'mask']), # add 1 dimension because we have only 1 channel
        monai.transforms.ScaleIntensityd(keys=['img'],minv=0, maxv=1) # scale intensity to values between 0-1
    ]
)
    

In [202]:
train_dict_list = np.load(os.path.join(data_path, 'file_list_train.npy'),allow_pickle=True)
test_dict_list = np.load(os.path.join(data_path, 'file_list_test.npy'),allow_pickle=True)

train_dataset = monai.data.CacheDataset(train_dict_list, transform=train_transform)
test_dataset = monai.data.CacheDataset(test_dict_list, transform=train_transform)

train_loader = monai.data.DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = monai.data.DataLoader(test_dataset, batch_size=10)

Loading dataset: 100%|██████████| 476/476 [00:04<00:00, 108.31it/s]


In [203]:
# Model parameters
model = monai.networks.nets.UNet(
    dimensions=2,
    in_channels=1,
    out_channels=1,
    channels = (48, 52, 104, 208, 416),
    strides=(1, 2, 2, 2),
    num_res_units=2,
).to(device)

loss_function =  monai.losses.DiceLoss(sigmoid=True, batch=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# logging to weights and biases

In [204]:
os.environ["WANDB_NOTEBOOK_NAME"] = "./ACDC_Segmentation.ipynb"
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [205]:
def wandb_masks(mask_output, mask_gt):
    """ Function that generates a mask dictionary in format that W&B requires """

    # Apply sigmoid to model ouput and round to nearest integer (0 or 1)
    sigmoid = torch.nn.Sigmoid()
    mask_output = sigmoid(mask_output)
    mask_output = torch.round(mask_output)

    # Transform masks to numpy arrays on CPU
    # Note: .squeeze() removes all dimensions with a size of 1 (here, it makes the tensors 2-dimensional)
    # Note: .detach() removes a tensor from the computational graph to prevent gradient computation for it
    mask_output = mask_output.squeeze().detach().cpu().numpy()
    mask_gt = mask_gt.squeeze().detach().cpu().numpy()

    # Create mask dictionary with class label and insert masks
    class_labels = {1: 'ribs'}
    masks = {
        'predictions': {'mask_data': mask_output, 'class_labels': class_labels},
        'ground truth': {'mask_data': mask_gt, 'class_labels': class_labels}
    }
    return masks

def log_to_wandb(epoch, train_loss, val_loss, batch_data, outputs):
    """ Function that logs ongoing training variables to W&B """

    # Create list of images that have segmentation masks for model output and ground truth
    log_imgs = [wandb.Image(img, masks=wandb_masks(mask_output, mask_gt)) for img, mask_output,
                mask_gt in zip(batch_data['img'], outputs, batch_data['mask'])]

    # Send epoch, losses and images to W&B
    wandb.log({'epoch': epoch, 'train_loss': train_loss, 'val_loss': val_loss, 'results': log_imgs})

In [206]:
def from_compose_to_list(transform_compose):
    """
    Transform an object monai.transforms.Compose in a list fully describing the transform.
    /!\ Random seed is not saved, then reproducibility is not enabled.
    """
    from copy import deepcopy
        
    if not isinstance(transform_compose, monai.transforms.Compose):
        raise TypeError("transform_compose should be a monai.transforms.Compose object.")
    
    output_list = list()
    for transform in transform_compose.transforms:
        kwargs = deepcopy(vars(transform))
        
        # Remove attributes which are not arguments
        args = list(transform.__init__.__code__.co_varnames[1: transform.__init__.__code__.co_argcount])
        for key, obj in vars(transform).items():
            if key not in args:
                del kwargs[key]

        output_list.append({"class": transform.__class__, "kwargs": kwargs})
    return output_list

def from_list_to_compose(transform_list):
    """
    Transform a list in the corresponding monai.transforms.Compose object.
    """
    
    if not isinstance(transform_list, list):
        raise TypeError("transform_list should be a list.")
    
    pre_compose_list = list()
    
    for transform_dict in transform_list:
        if not isinstance(transform_dict, dict) or 'class' not in transform_dict or 'kwargs' not in transform_dict:
            raise TypeError("transform_list should only contains dicts with keys ['class', 'kwargs']")
        
        try:
            transform = transform_dict['class'](**transform_dict['kwargs'])
        except TypeError: # Classes have been converted to str after saving
            transform = eval(transform_dict['class'].replace("__main__.", ""))(**transform_dict['kwargs'])
            
        pre_compose_list.append(transform)
        
    return monai.transforms.Compose(pre_compose_list)

# training the network

In [213]:
run = wandb.init(
    project='ACDC_segmentation',
    name='LV_test5',
    config={
        'loss function': str(loss_function), 
        'lr': optimizer.param_groups[0]["lr"],
        'transform': from_compose_to_list(train_transform),
        'batch_size': train_loader.batch_size,
    }
)
# Do not hesitate to enrich this list of settings to be able to correctly keep track of your experiments!
# For example you should information on your model...
run_id = run.id # We remember here the run ID to be able to write the evaluation metrics

val_loss_best = 1e10
n_epochs = 200
os.mkdir(os.path.join('trained_nets', f'{run.name}'))

for epoch in tqdm(range(n_epochs)):
    model.train()    
    epoch_loss = 0
    step = 0
    for batch_data in train_loader: 
        step += 1
        optimizer.zero_grad()
        outputs = model(batch_data["img"].float().to(device))
        loss = loss_function(outputs, batch_data["mask"].to(device))
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    train_loss = epoch_loss/step
    #print("tl", train_loss)

    # validation part
    step = 0
    val_loss = 0
    for batch_data in test_loader:
        step += 1
        model.eval()
        outputs = model(batch_data['img'].float().to(device))
        loss = loss_function(outputs, batch_data['mask'].to(device))
        val_loss+= loss.item()
    val_loss = val_loss / step
    #print("vl", train_loss)
    log_to_wandb(epoch, train_loss, val_loss, batch_data, outputs)

# Store the network parameters        
    if val_loss < val_loss_best:  
        torch.save(model.state_dict(), f'trained_nets/{run.name}/trainedUNet_{run.name}.pt')
        val_loss_best = val_loss
run.finish()

# save the train and test lists
np.save(f'trained_nets/{run.name}/test_dict_list_{run.name}',test_dict_list)
np.save(f'trained_nets/{run.name}/train_dict_list_{run.name}',train_dict_list)

Traceback (most recent call last):
  File "/home/jovyan/.local/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 996, in init
    wi.setup(kwargs)
  File "/home/jovyan/.local/lib/python3.8/site-packages/wandb/sdk/wandb_init.py", line 168, in setup
    tel.feature.set_init_tags = True
  File "/home/jovyan/.local/lib/python3.8/site-packages/wandb/sdk/lib/telemetry.py", line 41, in __exit__
    self._run._telemetry_callback(self._obj)
  File "/home/jovyan/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 577, in _telemetry_callback
    self._telemetry_flush()
  File "/home/jovyan/.local/lib/python3.8/site-packages/wandb/sdk/wandb_run.py", line 588, in _telemetry_flush
    self._backend.interface._publish_telemetry(self._telemetry_obj)
  File "/home/jovyan/.local/lib/python3.8/site-packages/wandb/sdk/interface/interface_shared.py", line 74, in _publish_telemetry
    self._publish(rec)
  File "/home/jovyan/.local/lib/python3.8/site-packages/wandb/sdk/interface/interf

Exception: problem